## Get all imports

In [1]:
import math
import numpy as np
import pandas as pd
from collections import Counter
from PyImpetus import PPIMBR
from sklearn.svm import LinearSVR, SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import Ridge
from lightgbm import LGBMRegressor
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import time

## Load data

In [2]:
# Load the data into a dataframe
df = pd.read_csv("slice_localization_data.csv")
# Pre-process the data
df = df.drop(["patientId"], axis=1)
# Lets check it out
display(df.head())
# Split the data into input features and target variable
data, Y = df.drop(["reference"], axis=1), df["reference"].values
# Lets check out the shape of our data
print("Data shape: ", data.shape, "Target Variable shape: ", Y.shape)

,value0,value1,value2,value3,value4,value5,value6,value7,value8,value9,...,value375,value376,value377,value378,value379,value380,value381,value382,value383,reference
0,0.0,0.0,0.0,0.0,0.0,0.0,-0.25,-0.25,-0.25,-0.25,...,-0.25,0.980381,0.0,0.0,0.0,0.0,0.0,-0.25,-0.25,21.803851
1,0.0,0.0,0.0,0.0,0.0,0.0,-0.25,-0.25,-0.25,-0.25,...,-0.25,0.977008,0.0,0.0,0.0,0.0,0.0,-0.25,-0.25,21.745726
2,0.0,0.0,0.0,0.0,0.0,0.0,-0.25,-0.25,-0.25,-0.25,...,-0.25,0.977008,0.0,0.0,0.0,0.0,0.0,-0.25,-0.25,21.687600
3,0.0,0.0,0.0,0.0,0.0,0.0,-0.25,-0.25,-0.25,-0.25,...,-0.25,0.977008,0.0,0.0,0.0,0.0,0.0,-0.25,-0.25,21.629474
4,0.0,0.0,0.0,0.0,0.0,0.0,-0.25,-0.25,-0.25,-0.25,...,-0.25,0.976833,0.0,0.0,0.0,0.0,0.0,-0.25,-0.25,21.571348


Data shape:  (53500, 384) Target Variable shape:  (53500,)


## Modelling with Decision Tree without PyImpetus

In [4]:
# Use Train-test split for understanding the performance of PyImpetus
x_train, x_test, y_train, y_test = train_test_split(data, Y, test_size=0.2, random_state=27, shuffle=True)
# We want to time our algorithm
start = time.time()

# Convert the data into numpy arrays
x_train, x_test = x_train.values, x_test.values

model = DecisionTreeRegressor(random_state=27)
model.fit(x_train, y_train)
preds = model.predict(x_test)
score = mean_squared_error(y_test, preds)
print("MSE: ", score)
end = time.time()
print("Time Required (in seconds): ", end-start)

MSE:  5.975008446757659
Time Required (in seconds):  5.493305921554565


## Modelling with Decision Tree using PyImpetus (fast but less robust feature subset selection)

In [7]:
# Use Train-test split for understanding the performance of PyImpetus
x_train, x_test, y_train, y_test = train_test_split(data, Y, test_size=0.2, random_state=27, shuffle=True)
# We want to time our algorithm
start = time.time()

# Create a PyImpetus classification object and initialize with required parameters
# NOTE: To achieve fast selection, set cv=0 for disabling the use of any internal cross-validation
model = PPIMBR(DecisionTreeRegressor(random_state=27), cv=0, p_val_thresh=0.001, num_simul=5, random_state=27, verbose=0)
# Fit this above object on the train part and transform the train dataset into selected feature subset
# NOTE: x_train has to be a dataframe and y_train has to be a numpy array
x_train = model.fit_transform(x_train, y_train)
# Transform the test set as well
# NOTE: x_test has to be a dataframe
x_test = model.transform(x_test)
# Check out the number of features selected
print("Number of features selected: ", len(model.MB))
# Convert the data into numpy arrays
x_train, x_test = x_train.values, x_test.values

model = DecisionTreeRegressor(random_state=27)
model.fit(x_train, y_train)
preds = model.predict(x_test)
score = mean_squared_error(y_test, preds)
print("Score: ", score)
end = time.time()
print("Time Required (in seconds): ", end-start)

Number of features selected:  45
Score:  5.16066413147826
Time Required (in seconds):  1296.1260921955109


# Final Results (MSE is used so less is better)

### Final MSE without PyImpetus: 5.98 (5.49 seconds)
### Final MSE with PyImpetus (cv==0): 5.16066413147826 (1296.13 seconds)